In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import sys

In [ ]:
from models.seir.seir_movement import SEIR_Movement
from utils.generic import compute_regionwiseflow

In [ ]:
regionwiseflow, districts = compute_regionwiseflow()
regionwiseflow

In [ ]:
pd.plotting.register_matplotlib_converters()
for district in districts:
    plt.figure(figsize=(12, 12))
    if district == 'Murbad':
        continue
    plt.plot(regionwiseflow.loc[district, :].index, regionwiseflow.loc[district, 'mu'])
    plt.title('Net Flow/population for {}'.format(district))
    plt.xlabel('Date')
    plt.ylabel('Net Flow/population')
    plt.show()

In [ ]:
R0 = 2.2 
T_inf = 2.9
T_trans = T_inf/R0
T_inc = 5.2
T_recov_mild = (14 - T_inf)
T_hosp = 5
T_recov_severe = (31.5 - T_inf)
T_death = 32
 
P_severe = 0.2
P_fatal = 0.02
P_mild = 1 - P_severe - P_fatal

N = 7e6
I0 = 1.0

# new_R0 = 0.74
# intervention_amount = R0/new_R0
intervention_day = 100
intervention_amount = 3

mu = 0.003

params = [mu, T_trans, T_inc, T_inf, T_recov_mild, T_hosp, T_recov_severe, T_death, 
          P_mild, P_severe, P_fatal, N, intervention_day, intervention_amount]

# S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
state_init_values = [(N - I0)/N, 0, I0/N, 0, 0, 0, 0, 0, 0]

In [ ]:
solver = SEIR_Movement(params, state_init_values)
sol = solver.solve_ode(time_step=1)
states_int_array = (sol.y*N).astype('int')

In [ ]:
E = states_int_array[1]
I = states_int_array[2]
H = states_int_array[5]
F = states_int_array[8]

In [ ]:
ind = sol.t    # the x locations for the groups
width = 0.95        # the width of the bars: can also be len(x) sequence

plt.figure(figsize=(12, 12))
p1 = plt.bar(ind, F, width)
p2 = plt.bar(ind, H, width, bottom=F)
p3 = plt.bar(ind, I, width, bottom=H+F)
p4 = plt.bar(ind, E, width, bottom=I+H+F)
plt.ylabel('No of people')
plt.legend((p1[0], p2[0], p3[0], p4[0]), ('Dead', 'Hospitalised', 'Infected', 'Exposed'))

plt.show()